In [1]:
import numpy as np
import pandas as pd

$\frac{dS^i}{S^i} = \sum_{l} \beta_{i,l} \frac{dI^l}{I^l} + \alpha_i + dZ^i$

In [2]:
df = pd.read_csv('sp500_data.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.set_index('date',inplace=True)

In [3]:
# 442 stock returns over 5 years - 1258 days
df.shape

(1259, 442)

In [4]:
# get rid of _price
col_list = []
for i in df.columns:
    col_list.append(i.split('_', 1)[0])
df.columns = col_list

In [5]:
df = df.pct_change()

In [6]:
df = df.dropna(axis=0)

In [7]:
import glob

path = '/Users/zachariemartin/Desktop/School/Projects/summer2019/sum19/etf data'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df_ = pd.read_csv(filename, usecols=[4], header='infer')
    li.append(df_)

etf_df = pd.concat(li, axis=1, ignore_index=True)

In [8]:
etf_df = etf_df.pct_change()
etf_df = etf_df.dropna(axis=0)

In [9]:
# 9 etf returns over 5 years - 1258 days

x = etf_df.values
x.shape

(1258, 9)

In [10]:
from sklearn import linear_model

# regress on each stocks returns, save residuals
reg = linear_model.LinearRegression()
x = etf_df.values
df_residuals = pd.DataFrame(columns=df.columns)

for i in range(df.shape[1]):
    y = df.iloc[:,i].values.reshape(-1,1)
    reg.fit(x,y)
    w = reg.coef_
    b = reg.intercept_
    df_residuals.iloc[:,i] = (y-(x@w.T + b)).flatten()

In [35]:
df_residuals = df_residuals.cumsum()

In [37]:
from statsmodels.tsa.stattools import adfuller

df_df = pd.DataFrame(columns=['ADF Statistic','p-value','n lags used'],index=df_residuals.columns)

# test residuals for stationarity and return adf statistic and p-value, columns 0 and 1 respectively
for i in range(df_residuals.shape[1]):
    result = adfuller(df.iloc[:,i])
    df_df.iloc[i,0] = result[0]
    df_df.iloc[i,1] = result[1]
    df_df.iloc[i,2] = result[2]


In [38]:
# check for significance at 5%
np.where(df_df.iloc[:,1] > .05)

(array([], dtype=int64),)

In [39]:
df_df

ADF Statistic      p-value n lags used
AAPL       -10.8282  1.73499e-19           8
ACN        -26.5612            0           1
AMG        -34.0833            0           0
AMT        -13.8091  8.29393e-26           8
ANTM       -36.7231            0           0
BBBY       -35.7298            0           0
BK         -34.7934            0           0
CAH         -36.784            0           0
DE         -35.4293            0           0
DLTR       -16.9549  9.40131e-30           3
ETFC       -35.6518            0           0
ETN        -18.4407  2.16132e-30           3
FLS        -34.4618            0           0
FMC        -34.7624            0           0
FOSL       -35.0349            0           0
INTU       -18.8436            0           3
NAVI        -34.822            0           0
NDAQ       -18.9383            0           3
NLSN       -26.1485            0           1
NOC        -15.5925  1.87252e-28           6
NWSA       -35.9671            0           0
PCG        -9.11736  3.27496e-15          16
PFE        -33.8822            0           0
PFG        -35.4372            0           0
PRGO       -26.6934            0           1
R          -25.8422            0           1
ROP        -37.4574            0           0
SBAC       -14.3979  8.60646e-27           8
SCHW       -19.0064            0           2
SWKS       -36.2174            0           0
...             ...          ...         ...
ZTS        -39.1054            0           0
ADM         -34.814            0           0
AZO        -38.2355            0           0
C          -34.6396            0           0
CCI        -14.1462  2.20195e-26           8
CME        -36.7317            0           0
CMS        -13.2353  9.40577e-25           8
DG         -26.7515            0           1
DVA         -35.806            0           0
ENDP       -18.9596            0           2
EQIX       -19.7566            0           3
FITB       -35.4763            0           0
GOOGL      -14.7429  2.55468e-27           7
HON        -37.2175            0           0
IBM        -35.4749            0           0
KSS         -36.776            0           0
LBTYK      -14.2281  1.61413e-26           7
LYB        -25.7994            0           1
MA         -18.2076  2.39715e-30           4
MDT        -19.4826            0           3
MLM        -25.7288            0           1
MSFT       -15.0298  9.94007e-28           7
NFLX       -16.5437  1.97338e-29           4
O          -35.3618            0           0
PCAR       -36.4356            0           0
SRCL       -36.1383            0           0
TGNA       -10.4279  1.63931e-18          11
TXT        -35.0726            0           0
V          -18.2565  2.33637e-30           4
WU         -26.4304            0           1

[442 rows x 3 columns]